In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pandas_profiling
import missingno as msno
% matplotlib inline
% config InlineBackend.figure_format='retina'

sns.set()


In [3]:
# 保存subgroup
def save_subgroup(dataframe, g_index, subgroup_name, path, prefix="raw_"):
    save_subgroup_filename = path + prefix + subgroup_name + ".csv.gz"
    print("save filename:", save_subgroup_filename)
    dataframe.to_csv(save_subgroup_filename, compression="gzip", encoding="UTF-8")

    test_df = pd.read_csv(save_subgroup_filename, compression="gzip", index_col=g_index, encoding="UTF-8")

    if dataframe.equals(test_df):
        print("Test-passed: we recover the equivalent subgroup dataframe.")
    else:
        print("Warning -- equivalence test!!! Double-check.")


In [4]:
# 读取subgroup
def load_subgroup(filename, index_col=[0]):
    return pd.read_csv(filename, compression="gzip", index_col=index_col)


In [14]:
# 读取数据
clean_players = load_subgroup(
    "E:\Workspace\jupyter_notebook\\notebook_idata_lesson01\RedCard\data\\cleaned_players.csv.gz")
players = load_subgroup("E:\Workspace\jupyter_notebook\\notebook_idata_lesson01\RedCard\data\\raw_players.csv.gz")
countries = load_subgroup("E:\Workspace\jupyter_notebook\\notebook_idata_lesson01\RedCard\data\\raw_countries.csv.gz")
referees = load_subgroup("E:\Workspace\jupyter_notebook\\notebook_idata_lesson01\RedCard\data\\raw_referees.csv.gz")
agg_dyads = pd.read_csv("E:\Workspace\jupyter_notebook\\notebook_idata_lesson01\RedCard\data\\raw_dyads.csv.gz",
                        compression='gzip', index_col=[0, 1])


In [16]:
agg_dyads[:10]


<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th>games</th>
      <th>victories</th>
      <th>ties</th>
      <th>defeats</th>
      <th>goals</th>
      <th>yellowCards</th>
      <th>yellowReds</th>
      <th>redCards</th>
    </tr>
    <tr>
      <th>refNum</th>
      <th>playerShort</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <th>lucas-wilchez</th>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>2</th>
      <th>john-utaka</th>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">3</th>
      <th>abdon-prats</th>
      <td>1</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>pablo-mari</th>
      <td>1</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>ruben-pena</th>
      <td>1</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th rowspan="5" valign="top">4</th>
      <th>aaron-hughes</th>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>aleksandar-kolarov</th>
      <td>1</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>alexander-tettey</th>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>anders-lindegaard</th>
      <td>1</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>andreas-beck</th>
      <td>1</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>
</div>

In [17]:
# 验证原始数据是否正确
all(agg_dyads["games"] == agg_dyads["victories"] + agg_dyads["ties"] + agg_dyads["defeats"])


True

In [18]:
len(agg_dyads.reset_index().set_index('playerShort'))


146028

In [19]:
agg_dyads['totalRedCards'] = agg_dyads['yellowReds'] + agg_dyads['redCards']
agg_dyads.rename(columns={'redCards': 'strictRedCards'}, inplace=True)


In [20]:
agg_dyads.head()


<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th>games</th>
      <th>victories</th>
      <th>ties</th>
      <th>defeats</th>
      <th>goals</th>
      <th>yellowCards</th>
      <th>yellowReds</th>
      <th>strictRedCards</th>
      <th>totalRedCards</th>
    </tr>
    <tr>
      <th>refNum</th>
      <th>playerShort</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <th>lucas-wilchez</th>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>2</th>
      <th>john-utaka</th>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">3</th>
      <th>abdon-prats</th>
      <td>1</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>pablo-mari</th>
      <td>1</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>ruben-pena</th>
      <td>1</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>
</div>

In [21]:
clean_players.head()


,height,weight,skintone,position_agg,weightclass,heightclass,skintoneclass,age_years
playerShort,,,,,,,,
aaron-hughes,182.0,71.0,0.125,Defense,low_weight,mid_height,"(-0.001, 0.125]",33.149897
aaron-hunt,183.0,73.0,0.125,Forward,low_weight,mid_height,"(-0.001, 0.125]",26.327173
aaron-lennon,165.0,63.0,0.250,Midfield,vlow_weight,vlow_height,"(0.125, 0.25]",25.713895
aaron-ramsey,178.0,76.0,0.000,Midfield,mid_weight,low_height,"(-0.001, 0.125]",22.017796
abdelhamid-el-kaoutari,180.0,73.0,0.250,Defense,low_weight,low_height,"(0.125, 0.25]",22.795346


In [22]:
agg_dyads.reset_index().head()


,refNum,playerShort,games,victories,ties,defeats,goals,yellowCards,yellowReds,strictRedCards,totalRedCards
0,1,lucas-wilchez,1,0,0,1,0,0,0,0,0
1,2,john-utaka,1,0,0,1,0,1,0,0,0
2,3,abdon-prats,1,0,1,0,0,1,0,0,0
3,3,pablo-mari,1,1,0,0,0,0,0,0,0
4,3,ruben-pena,1,1,0,0,0,0,0,0,0


In [23]:
agg_dyads.reset_index().set_index('playerShort').head()


,refNum,games,victories,ties,defeats,goals,yellowCards,yellowReds,strictRedCards,totalRedCards
playerShort,,,,,,,,,,
lucas-wilchez,1,1,0,0,1,0,0,0,0,0
john-utaka,2,1,0,0,1,0,1,0,0,0
abdon-prats,3,1,0,1,0,0,1,0,0,0
pablo-mari,3,1,1,0,0,0,0,0,0,0
ruben-pena,3,1,1,0,0,0,0,0,0,0


In [24]:
# clean_players和agg_dyads合并
player_dyad = (clean_players.merge(agg_dyads.reset_index().set_index('playerShort'),
                                   left_index=True,
                                   right_index=True))


In [25]:
player_dyad.head()


,height,weight,skintone,position_agg,weightclass,heightclass,skintoneclass,age_years,refNum,games,victories,ties,defeats,goals,yellowCards,yellowReds,strictRedCards,totalRedCards
playerShort,,,,,,,,,,,,,,,,,,
aaron-hughes,182.0,71.0,0.125,Defense,low_weight,mid_height,"(-0.001, 0.125]",33.149897,4,1,0,0,1,0,0,0,0,0
aaron-hughes,182.0,71.0,0.125,Defense,low_weight,mid_height,"(-0.001, 0.125]",33.149897,66,1,1,0,0,0,0,0,0,0
aaron-hughes,182.0,71.0,0.125,Defense,low_weight,mid_height,"(-0.001, 0.125]",33.149897,77,26,13,8,5,0,0,0,0,0
aaron-hughes,182.0,71.0,0.125,Defense,low_weight,mid_height,"(-0.001, 0.125]",33.149897,163,2,1,1,0,0,0,0,0,0
aaron-hughes,182.0,71.0,0.125,Defense,low_weight,mid_height,"(-0.001, 0.125]",33.149897,194,16,3,5,8,0,2,0,0,0


In [28]:
clean_dyads = (
    agg_dyads.reset_index()[agg_dyads.reset_index()["playerShort"].isin(set(clean_players.index))]).set_index(
    ['refNum', 'playerShort'])

clean_dyads.head()


<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th>games</th>
      <th>victories</th>
      <th>ties</th>
      <th>defeats</th>
      <th>goals</th>
      <th>yellowCards</th>
      <th>yellowReds</th>
      <th>strictRedCards</th>
      <th>totalRedCards</th>
    </tr>
    <tr>
      <th>refNum</th>
      <th>playerShort</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <th>lucas-wilchez</th>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>2</th>
      <th>john-utaka</th>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">4</th>
      <th>aaron-hughes</th>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>aleksandar-kolarov</th>
      <td>1</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>alexander-tettey</th>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>
</div>

In [29]:
clean_dyads.shape, agg_dyads.shape, player_dyad.shape


((124621, 9), (146028, 9), (124621, 18))

In [30]:
colnames = ['games', 'totalRedCards']
j = 0
out = [0 for _ in range(sum(clean_dyads['games']))]

for index, row in clean_dyads.reset_index().iterrows():
    n = row['games']
    d = row['totalRedCards']
    ref = row['refNum']
    player = row['playerShort']
    for _ in range(n):
        row['totalRedCards'] = 1 if (d - _) > 0 else 0
        rowlist = list([ref, player, row['totalRedCards']])
        out[j] = rowlist
        j += 1

tidy_dyads = pd.DataFrame(out, columns=['refNum', 'playerShort', 'redcard'], ).set_index(['refNum', 'playerShort'])


In [32]:
print(tidy_dyads.shape)
tidy_dyads[:5]


(373067, 1)


<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th>redcard</th>
    </tr>
    <tr>
      <th>refNum</th>
      <th>playerShort</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <th>lucas-wilchez</th>
      <td>0</td>
    </tr>
    <tr>
      <th>2</th>
      <th>john-utaka</th>
      <td>0</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">4</th>
      <th>aaron-hughes</th>
      <td>0</td>
    </tr>
    <tr>
      <th>aleksandar-kolarov</th>
      <td>0</td>
    </tr>
    <tr>
      <th>alexander-tettey</th>
      <td>0</td>
    </tr>
  </tbody>
</table>
</div>

In [33]:
tidy_dyads.to_csv("E:\Workspace\jupyter_notebook\\notebook_idata_lesson01\RedCard\data\\cleaned_dyads.csv.gz",
                  compression='gzip')
